In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.utils.training_utils import multi_gpu_model

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import DenseNet201

Using TensorFlow backend.


In [2]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [3]:
#TOT = 35534421
TOT = 46613580 - 340000
EPOCHS = 70
size = 71
batchsize = 256
lw = 6
channel = 3
STEPS = TOT / EPOCHS / batchsize
NCATS = 340
mixup = 0
center = False
optimizer = 'adam'
cb = 'raw'

In [4]:
K.clear_session()
# model = Xception(input_shape=(size, size, channel), weights=None, classes=NCATS)
model = DenseNet201(input_shape=(size,size,3), weights=None, classes=NCATS)

if optimizer == 'sgd':
    opt = SGD(0.002, momentum=0.9, nesterov=True)
    opt = NormalizedOptimizer(opt, normalization='l2')
elif optimizer == 'adam':
    opt = Adam(lr=0.001)

# model = multi_gpu_model(model, gpus=2)

# model.load_weights('models/xception71_time_lw6_balance_0_adam.model')

model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 71, 71, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 77, 77, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 36, 36, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 36, 36, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [5]:
valid_df = pd.read_csv('../input/valid_raw.csv')
np_classes = np.load('../input/classes.npy')
cat2id = {cat.replace(' ', '_'):k for k, cat in enumerate(np_classes)}
valid_df.word = valid_df.word.apply(lambda x: cat2id[x.replace(' ', '_')])
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)
x_valid = df_to_image_array_temp(valid_df, size, draw_drawtime_parts, lw=lw, preprocess_input=preprocess_input)

/home/kownse/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/kownse/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


x shape (34000, 71, 71, 3) x max 255


In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 71, 71, 3) (34000, 340)
Validation array memory 1.92 GB


In [7]:
train_datagen = image_detail_generator_temp(size=size, 
                                            batchsize=batchsize,
                                            drawfunc = draw_drawtime_parts,
                                            lw=lw, 
                                           preprocess_input=preprocess_input)

In [8]:
model_prefix = 'dense201_{}_drawtime_part_lw{}_{}_{}'.format(size, lw, mixup, optimizer, cb)

print(model_prefix)
check_path = "./models/{}.model".format(model_prefix)
fold = 9
if cb == 'raw':
    callbks = [
        ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5,
                          min_delta=0.005, mode='max', cooldown=3, verbose=1),
        callbacks.ModelCheckpoint(check_path,
                                    monitor='val_top_3_accuracy', 
                                     mode = 'max', 
                                  save_best_only=True, save_weights_only=True,
                                  verbose=1),
        callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
        EarlyStopping(monitor='val_top_3_accuracy', mode='max',patience=20, verbose=1)
    ]
elif cb == 'snap':
    snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                       nb_snapshots=3,
                                       size = size,
                                       init_lr=1e-3, 
                                       fold=fold)
 
    callbks = snapshot.get_callbacks(model_prefix = model_prefix)

dense201_71_drawtime_part_lw6_0_adam


In [9]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks,
    workers = 1,
    max_queue_size=100,
    use_multiprocessing=False  
)

gen initedEpoch 1/70



ResourceExhaustedError: OOM when allocating tensor with shape[256,1376,2,2] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv5_block16_0_bn/FusedBatchNorm}} = FusedBatchNorm[T=DT_FLOAT, _class=["loc:@train...chNormGrad"], data_format="NCHW", epsilon=1.001e-05, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv5_block15_concat/concat, conv5_block16_0_bn/gamma/read, conv5_block16_0_bn/beta/read, bn/Const_4, bn/Const_4)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics/top_3_accuracy/Mean_1/_7705}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_64739_metrics/top_3_accuracy/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/ta

In [ ]:
test = pd.read_csv('../input/test_raw.csv')
test.head()
x_test = df_to_image_array_temp(test, size, draw_drawtime_parts, lw=lw, 
                              preprocess_input=preprocess_input)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}


In [ ]:
def doodle_predict(model, model_path, x_test):
    model.load_weights(model_path)

    test_predictions = model.predict(x_test, batch_size=128, verbose=1)
    np.save('../result/pre_{}.npy'.format(model_path))
    
    top3 = preds2catids(test_predictions)
    top3cats = top3.replace(id2cat)
    test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
    submission = test[['key_id', 'word']]

    import kaggle_util
    kaggle_util.save_result(submission,  
                            '../result/{}.csv'.format(model_prefix), 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False)

In [ ]:
doodle_predict(model, check_path, x_test)